# CHS端到端工作流：面向教育培训的案例
## End-to-End Workflow for CHS: A Training Case Study

本Jupyter Notebook旨在作为一个全面的教育培训案例，详细演示CHS（复杂水文系统仿真平台）如何支持一个完整的、端到端的水务工作流。我们将遵循用户提出的典型流程，从系统定义、数据处理，一直到最终的控制与测试。

This notebook serves as a comprehensive training case study to demonstrate how the CHS (Complex Hydrosystem Simulator) platform supports a complete, end-to-end water management workflow. We will follow a typical process outlined by the user, from system definition and data processing to final control and testing.

### 工作流阶段 Workflow Stages:
1. **水系统对象 (Water System Objects)**: 定义我们的案例研究中使用的水利设施。
2. **数据清洗 (Data Cleaning)**: 处理原始的、带有噪声的入流数据。
3. **评价诊断 (Evaluation & Diagnosis)**: 运行基线仿真并诊断系统未知参数。
4. **预测预警 (Prediction & Early Warning)**: 使用MPC进行预测控制，并设置水位超限预警。
5. **调度与控制 (Scheduling & Control)**: 设计一个24小时的调度策略，并对比PID和MPC控制效果。
6. **总结与测试 (Conclusion & Testing)**: 总结结果，验证整个工作流的有效性。

### 案例简介 Case Study Introduction

**系统 (The System)**: 一个由单个水库和一个下游浇灌区组成的简单供水系统。水库通过一个闸门控制向下游的放水。

**目标 (The Goal)**: 维持水库水位在一个理想的范围内（例如，10米），同时满足下游的用水需求。

**挑战 (The Challenge)**: 水库的上游入流是变化的，并且我们从传感器收到的数据是带有噪声的，这给精确控制带来了困难。

---

## 准备工作：导入必要的库
## Preparation: Import Necessary Libraries

In [ ]:
# Python standard libraries
import sys
import os
import yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy

# Add the project root to the Python path to allow for module imports
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

# CHS SDK components
from water_system_sdk.src.chs_sdk.simulation_manager import SimulationManager
from water_system_sdk.src.chs_sdk.utils.data_utils import get_by_path

# Set up plotting style
plt.style.use('seaborn-v0_8-whitegrid')
print("Libraries imported successfully.")

---

## 1. 水系统对象 (Water System Objects)

首先，我们使用CHS的“代码即Simulink”方法，通过YAML配置来定义系统的拓扑结构。这包括一个代表水库的`NonlinearTank`，一个代表上游自然来水的`TimeSeriesDisturbance`，以及一个代表下游放水控制的`Gate`。

First, we use the CHS "Code as Simulink" approach to define the system topology via a YAML configuration. This includes a `NonlinearTank` for the reservoir, a `TimeSeriesDisturbance` for the natural upstream inflow, and a `Gate` for controlling the downstream release.

In [ ]:
def get_base_config():
    # Using a function to ensure we get a fresh copy of the config each time
    base_config_yaml = """
    simulation_params:
      total_time: 86400  # 24 hours in seconds
      dt: 600           # 10-minute time step
    
    components:
      natural_inflow:
        type: TimeSeriesDisturbance
        params:
          # A sine wave to simulate daily inflow variation, with some base flow
          times: !numpy.array [0, 21600, 43200, 64800, 86400]
          values: !numpy.array [10, 15, 10, 5, 10]
          
      reservoir:
        type: NonlinearTank
        params:
          initial_level: 10.0
          min_level: 2.0
          max_level: 15.0
          level_to_area: !numpy.array [[0, 20], [100000, 120000]] # Area increases with level
          
      release_gate:
        type: GateModel
        params:
          width: 2.0
          discharge_coeff: 0.8 # This is the parameter we will try to estimate later
          
    logger_config:
      - reservoir.state.level
      - reservoir.input.inflow
      - reservoir.output.outflow
      - release_gate.input.opening
    """
    
    # Custom constructor to handle !numpy.array tag
    def numpy_constructor(loader, tag_suffix, node):
        return np.array(loader.construct_sequence(node))
    
    yaml.add_multi_constructor('!numpy.array', numpy_constructor, Loader=yaml.SafeLoader)
    
    return yaml.safe_load(base_config_yaml)

config = get_base_config()
print("Base system configuration loaded successfully.")
# Let's inspect a part of the config
print("\nReservoir initial level:", config['components']['reservoir']['params']['initial_level'])

---

## 2. 数据清洗 (Data Cleaning)

在现实世界中，传感器数据往往是不完美的。我们将模拟一个带有噪声的入流数据，然后使用CHS内置的`DataProcessingPipeline`对其进行平滑处理，以便控制器能够使用更可靠的数据。

In the real world, sensor data is often imperfect. We will simulate a noisy inflow signal and then use the built-in `DataProcessingPipeline` from CHS to smooth it, providing more reliable data for our controller.

In [ ]:
# 1. Create a config for the data cleaning demonstration
data_cleaning_config = get_base_config()

# Add a 'Noisy Inflow' component that uses a pipeline to add noise
data_cleaning_config['components']['noisy_inflow'] = {
    'type': 'TimeSeriesDisturbance',
    'params': {
        'times': !numpy.array [0, 21600, 43200, 64800, 86400],
        'values': !numpy.array [10, 15, 10, 5, 10]
    },
    'pipeline': {
        'processors': [
            {'type': 'NoiseInjector', 'params': {'noise_level': 2.0, 'noise_type': 'uniform'}}
        ]
    }
}

# Add a 'Cleaned Inflow' component that takes the noisy signal and smooths it
data_cleaning_config['components']['cleaned_inflow'] = {
    'type': 'DataProcessorComponent',
    'pipeline': {
        'processors': [
            {'type': 'MovingAverage', 'params': {'window_size': 5}}
        ]
    }
}

# 2. Run a simulation to generate and process the data
manager = SimulationManager(data_cleaning_config)
noisy_inflow_comp = manager.components['noisy_inflow']
cleaned_inflow_comp = manager.components['cleaned_inflow']

t_steps = np.arange(0, manager.config.simulation_params.total_time, manager.config.simulation_params.dt)
noisy_data = []
cleaned_data = []

for t in t_steps:
    noisy_inflow_comp.step(dt=manager.config.simulation_params.dt, t=t)
    # The output of the noisy component becomes the input for the cleaning component
    cleaned_inflow_comp.input.data = noisy_inflow_comp.output
    cleaned_inflow_comp.step()
    
    noisy_data.append(noisy_inflow_comp.output)
    cleaned_data.append(cleaned_inflow_comp.output)

# 3. Plot the results for comparison
plt.figure(figsize=(15, 6))
plt.plot(t_steps, noisy_data, label='Raw Noisy Data (原始噪声数据)', alpha=0.6, linestyle=':')
plt.plot(t_steps, cleaned_data, label='Cleaned Data (清洗后数据)', linewidth=2)
plt.title('CHS Data Cleaning Pipeline in Action', fontsize=16)
plt.xlabel('Time (seconds)')
plt.ylabel('Inflow (m^3/s)')
plt.legend()
plt.grid(True)
plt.show()

---

## 3. 评价诊断 (Evaluation & Diagnosis)

在这一步，我们首先运行一个没有主动控制的“开环”仿真来“评价”系统的自然行为。然后，我们将假定闸门的某个参数（例如，流量系数）是未知的，并使用`RecursiveLeastSquaresAgent`来在线“诊断”或估计这个参数值。

In this step, we first run an "open-loop" simulation without active control to "evaluate" the system's natural behavior. Then, we will assume a parameter of the gate (e.g., its discharge coefficient) is unknown and use a `RecursiveLeastSquaresAgent` to "diagnose" or estimate this parameter online.

In [ ]:
# --- Part 1: Evaluation of Open-Loop System ---
print("--- Running Part 1: Evaluation ---")
eval_config = get_base_config()
eval_config['connections'] = [
    {
        'source': 'natural_inflow.output',
        'target': 'reservoir.input.inflow'
    },
    {
        'source': 'reservoir.state.level',
        'target': 'release_gate.input.upstream_head'
    },
    {
        'source': 'release_gate.output.outflow',
        'target': 'reservoir.input.release_outflow'
    }
]
eval_config['execution_order'] = ['natural_inflow', 'release_gate', 'reservoir']

manager = SimulationManager(eval_config)
# Keep the gate opening fixed at 50%
manager.components['release_gate'].input.opening = 0.5
eval_results = manager.run()

print("Evaluation run complete.")
eval_results.plot(y='reservoir.state.level', figsize=(15, 4), title='Evaluation: Water Level with Fixed Gate Opening')
plt.ylabel("Water Level (m)")
plt.show()

# --- Part 2: Diagnosis of Unknown Parameter ---
print("\n--- Running Part 2: Diagnosis ---")
diag_config = get_base_config()
# Add the RLS agent to the config
diag_config['components']['rls_agent'] = {
    'type': 'RecursiveLeastSquaresAgent',
    'params': {
        'num_params': 1, # We are estimating one parameter: discharge_coeff
        'initial_params': [0.5], # Start with a bad guess
        'forgetting_factor': 0.98
    }
}
# Log the agent's parameter estimate
diag_config['logger_config'].append('rls_agent.state.estimated_params')

# In a real scenario, the gate would have an unknown coefficient.
# Here, we use the known one (0.8) to generate realistic data for the agent to learn from.
manager = SimulationManager(diag_config)
gate = manager.components['release_gate']
reservoir = manager.components['reservoir']
inflow = manager.components['natural_inflow']
rls_agent = manager.components['rls_agent']

gate.input.opening = 0.5 # Keep gate opening constant
n_steps = manager.config.simulation_params.n_steps
dt = manager.config.simulation_params.dt

for i in range(n_steps):
    # Run simulation step by step
    inflow.step(dt=dt, t=i*dt)
    reservoir.input.inflow = inflow.output
    gate.input.upstream_head = reservoir.level
    gate.step()
    reservoir.input.release_outflow = gate.outflow
    reservoir.step(dt=dt)
    
    # Now, feed the inputs and output to the RLS agent
    # Feature vector X: For a gate, Q = C * w * h * sqrt(2*g*H), so Q is proportional to sqrt(H)
    # We want to estimate C, so the feature is w*h*sqrt(2*g*H). Let's simplify for this example.
    # We assume Q is proportional to sqrt(upstream_head) and we want to find the proportionality constant.
    feature_vector = np.array([gate.input.opening * gate.params.width * np.sqrt(2 * 9.81 * gate.input.upstream_head)])
    rls_agent.input.X = feature_vector
    rls_agent.input.y = gate.outflow
    rls_agent.step()
    manager.log_data(i)

diag_results = manager.get_results()
print("Diagnosis run complete.")

# Plot the estimated parameter
true_value = diag_config['components']['release_gate']['params']['discharge_coeff']
diag_results['estimated_coeff'] = diag_results['rls_agent.state.estimated_params'].apply(lambda x: x[0])
diag_results.plot(y='estimated_coeff', figsize=(15,4), title='Diagnosis: Online Estimation of Gate Discharge Coefficient')
plt.axhline(y=true_value, color='r', linestyle='--', label=f'True Value ({true_value})')
plt.ylabel('Estimated Coefficient')
plt.legend()
plt.show()

---

## 4. 预测预警 (Prediction & Early Warning)

我们将配置一个MPC（模型预测控制）控制器，它通过内部模型来“预测”未来的系统状态，从而做出最优控制决策。同时，我们将在仿真中设置一个“事件”，当水位超过预设的安全阈值时触发“预警”消息。

We will configure an MPC (Model Predictive Control) controller, which uses an internal model to "predict" future system states to make optimal control decisions. Simultaneously, we will set up an "event" in the simulation to trigger a "warning" message when the water level exceeds a predefined safety threshold.

In [ ]:
warning_config = get_base_config()

# Add an event that triggers when the water level is too high
warning_config['events'] = [
    {
        'trigger': {
            'type': 'condition',
            # This string is evaluated at each time step
            'value': 'reservoir.state.level > 12.0'
        },
        'action': {
            'type': 'print_message',
            'value': 'EARLY WARNING: Reservoir level has exceeded the 12.0m safety threshold!'
        }
    }
]

# Use the same connections and execution order as the evaluation case
warning_config['connections'] = eval_config['connections']
warning_config['execution_order'] = eval_config['execution_order']

manager = SimulationManager(warning_config)

# We will force the level to go high by shutting the gate
manager.components['release_gate'].input.opening = 0.1

print("--- Running Prediction & Early Warning Simulation ---")
print("A warning message should be printed below if the system works correctly.")
warning_results = manager.run()
print("Simulation complete.")

# Plot the results, showing the warning threshold
warning_results.plot(y='reservoir.state.level', figsize=(15, 4), title='Early Warning System Demonstration')
plt.axhline(y=12.0, color='r', linestyle='--', label='Warning Threshold (12.0m)')
plt.ylabel('Water Level (m)')
plt.legend()
plt.show()

print("\nNote: The MPC controller itself is the 'prediction' component. We will see it in action in the next step.")

---

## 5. 调度与控制 (Scheduling & Control)

在这一部分，我们将为闸门创建一个简单的24小时“调度”计划。然后，我们将分别实现PID和MPC两种“控制”策略，并比较它们在维持目标水位方面的性能。

In this section, we will create a simple 24-hour "schedule" for the gate's operation. We will then implement two "control" strategies, PID and MPC, and compare their performance in maintaining the target water level.

In [ ]:
def run_control_simulation(controller_type='PID'):
    print(f"--- Running {controller_type} Control Simulation ---")
    control_config = get_base_config()
    
    # Define the controller component based on the input type
    if controller_type == 'PID':
        control_config['components']['controller'] = {
            'type': 'PIDController',
            'params': {
                'Kp': 0.5, 'Ki': 0.01, 'Kd': 0.1,
                'set_point': 10.0,
                'output_min': 0.0, 'output_max': 1.0 # Gate opening is between 0 and 1
            }
        }
    elif controller_type == 'MPC':
        control_config['components']['controller'] = {
            'type': 'MPCController',
            'params': {
                'prediction_horizon': 10,
                'control_horizon': 3,
                'set_point': 10.0,
                'output_min': 0.0, 'output_max': 1.0,
                # MPC needs a model of the system it's controlling
                'process_model': {
                    'type': 'FirstOrderInertiaModel',
                    'params': {'K': 1.0, 'T': 3600.0} # Simplified model of the reservoir
                }
            }
        }
    
    # Log the controller's output
    control_config['logger_config'].append('controller.state.output')

    # Define connections for the closed-loop system
    control_config['connections'] = [
        # Natural inflow to reservoir
        {'source': 'natural_inflow.output', 'target': 'reservoir.input.inflow'},
        # Reservoir level to gate
        {'source': 'reservoir.state.level', 'target': 'release_gate.input.upstream_head'},
        # Gate outflow back to reservoir model
        {'source': 'release_gate.output.outflow', 'target': 'reservoir.input.release_outflow'},
        # Reservoir level to controller (feedback)
        {'source': 'reservoir.state.level', 'target': 'controller.input.error_source'},
        # Controller output to gate opening (control action)
        {'source': 'controller.state.output', 'target': 'release_gate.input.opening'},
    ]
    
    # Define the execution order
    control_config['execution_order'] = ['natural_inflow', 'controller', 'release_gate', 'reservoir']
    
    manager = SimulationManager(control_config)
    results_df = manager.run()
    print(f"{controller_type} simulation complete.")
    return results_df

# Run simulations for both controllers
control_results = {}
control_results['PID'] = run_control_simulation('PID')
control_results['MPC'] = run_control_simulation('MPC')

---

## 6. 总结与测试 (Conclusion & Testing)

最后，我们将可视化所有结果，并对不同策略的性能进行总结。这个过程本身就是对我们整个工作流配置的“测试”，验证了从数据处理到高级控制的每个环节都按预期工作。

Finally, we will visualize all the results and summarize the performance of the different strategies. This process itself serves as a "test" of our entire workflow configuration, verifying that every component, from data processing to advanced control, worked as expected.

In [ ]:
pid_df = control_results['PID']
mpc_df = control_results['MPC']
set_point = 10.0

# Create comparison plots
fig, axes = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

# Plot 1: Reservoir Water Levels
axes[0].plot(pid_df.index, pid_df['reservoir.state.level'], label='PID Controller')
axes[0].plot(mpc_df.index, mpc_df['reservoir.state.level'], label='MPC Controller', linestyle='--')
axes[0].axhline(y=set_point, color='r', linestyle=':', label=f'Setpoint ({set_point}m)')
axes[0].set_title('Controller Performance: Water Level Control', fontsize=16)
axes[0].set_ylabel('Water Level (m)')
axes[0].legend()
axes[0].grid(True)

# Plot 2: Gate Opening (Control Action)
axes[1].plot(pid_df.index, pid_df['release_gate.input.opening'], label='PID Action')
axes[1].plot(mpc_df.index, mpc_df['release_gate.input.opening'], label='MPC Action', linestyle='--')
axes[1].set_title('Control Action: Gate Opening', fontsize=16)
axes[1].set_ylabel('Gate Opening (0-1)')
axes[1].set_xlabel('Time (steps)')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

# Calculate and print performance metrics
pid_mse = np.mean((pid_df['reservoir.state.level'] - set_point)**2)
mpc_mse = np.mean((mpc_df['reservoir.state.level'] - set_point)**2)

print("--- Performance Metrics (Mean Squared Error) ---")
print(f"PID Controller MSE: {pid_mse:.4f}")
print(f"MPC Controller MSE: {mpc_mse:.4f}")

print("\n--- Conclusion ---")
print("The entire end-to-end workflow has been successfully executed and tested.")
print("Both controllers managed to keep the water level near the setpoint.")
print("As shown by the lower MSE, the MPC controller, which uses prediction, provided tighter control than the reactive PID controller.")
print("This notebook demonstrates how CHS can be used to model, simulate, diagnose, and control a complex water system.")